In [29]:
import nlpcda
from nlpcda.tools.Basetool import Basetool
import jieba

In [37]:
class WordPositionExchange(Basetool):
    '''随机词语交换。'''
    
    from nlpcda.tools.Basetool import Basetool
    import jieba
    
    def __init__(self, create_num: int = 5, change_rate: float = 0.05, char_gram: int = 3, seed: int = 1):
        super(WordPositionExchange, self).__init__('', create_num, change_rate, seed)
        self.char_gram = char_gram

    def __replace_one(self, one_sentence: str):
        # 变为字 数组
        # sen_chars = list(one_sentence)
        sen_chars = list(jieba.cut(one_sentence, cut_all=False))
        for i in range(len(sen_chars)):
            if self.random.random() < self.change_rate:
                # 非中文字不动！
                if self.__is_chinese(sen_chars[i]) == False:
                    continue
                # 交换位置
                change_i = self.__cpt_exchange_position(sen_chars, i)
                # 进行交换
                sen_chars[i], sen_chars[change_i] = sen_chars[change_i], sen_chars[i]
        return ''.join(sen_chars)

    def __cpt_exchange_position(self, sen_chars: list, position_i):
        # 计算出交换位置
        i = position_i
        j = position_i
        # 从position_i左边，找到第一个不是中文的位置，or 全是中文则不能超过char_gram范围
        while i > 0 and self.__is_chinese(sen_chars[i]) and abs(i - position_i) < self.char_gram:
            i -= 1
        # 从position_i右边，找到第一个不是中文的位置，or 全是中文则不能超过char_gram范围
        while j < len(sen_chars) - 1 and self.__is_chinese(sen_chars[j]) and abs(j - position_i) < self.char_gram:
            j += 1
        # 不是中文导致的推出，需要撤回位置
        if not self.__is_chinese(sen_chars[i]):
            if i < position_i:
                i += 1
        if not self.__is_chinese(sen_chars[j]):
            if j > position_i:
                j -= 1
        return self.random.randint(i, j)

    def __is_chinese(self, a_chr):
        return u'\u4e00' <= a_chr <= u'\u9fff'

    def replace(self, replace_str: str):
        replace_str = replace_str.replace('\n', '').strip()
        sentences = [replace_str]
        t = 0

        while len(sentences) < self.create_num:
            t += 1
            a_sentence = self.__replace_one(replace_str)

            if a_sentence not in sentences:
                sentences.append(a_sentence)
            if t > self.create_num * self.loop_t / self.change_rate:
                break
        return sentences

In [42]:
exc = WordPositionExchange(char_gram=5)

x = '能否解决腐败问题，能否控制人口增长，能否消除两岸隔阂，都是实现中华民族伟大复兴的重要保证。\n江苏卫视《一站到底》节目首次引入的人工智能答题机器人“汪仔”，是由搜狐历时9个月，投资约4000多万元打造而成的。'
exc.replace(x)

['能否解决腐败问题，能否控制人口增长，能否消除两岸隔阂，都是实现中华民族伟大复兴的重要保证。江苏卫视《一站到底》节目首次引入的人工智能答题机器人“汪仔”，是由搜狐历时9个月，投资约4000多万元打造而成的。',
 '能否解决腐败问题，能否控制人口增长，隔阂消除两岸能否，都是实现中华民族保证的复兴重要伟大。江苏卫视《一站到底》节目首次引入的人工智能答题机器人“汪仔”，是由搜狐历时9个月，投资约4000多万元打造而成的。',
 '能否解决腐败问题，能否控制人口增长，能否消除两岸隔阂，都是实现复兴伟大中华民族的重要保证。江苏卫视《一站到底》答题首次引入的人工智能节目机器人“汪仔”，是由搜狐历时9个月，投资约4000多万元打造而成的。',
 '能否解决腐败问题，能否控制人口增长，能否消除两岸隔阂，都是实现中华民族伟大复兴的重要保证。江苏卫视《到底一站》节目首次引入的人工智能答题机器人“汪仔”，历时由搜狐是9个月，投资约4000多万元打造而成的。',
 '能否解决腐败问题，能否控制人口增长，能否消除两岸隔阂，都是实现中华民族伟大复兴的重要保证。江苏卫视《一站到底》节目的引入首次人工智能答题机器人“汪仔”，是由搜狐历时9个月，约投资4000多万元打造而成的。']